In [ ]:
import pandas as pd
import numpy as np
import requests
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from tqdm import tqdm
import shap

In [ ]:
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv', usecols = ['userId', 'movieId', 'rating'])
tags = pd.read_csv('ml-latest-small/tags.csv', usecols = ['userId', 'movieId', 'tag'])
links = pd.read_csv('ml-latest-small/links.csv')

In [ ]:
g = {i:lambda x: '|'.join(list(x)) for i in tags.columns[2:]}
tags = tags.groupby(['userId', 'movieId']).agg(g).reset_index()
movies = pd.merge(movies, links, on=['movieId'])
reviews = pd.merge(ratings, tags, on=['userId', 'movieId'])

In [ ]:
print("Movies head:\n", movies.head())
print("Reviews head:\n", reviews.head())

In [ ]:
movies_info = (movies.shape[1], movies.shape[0])
ratings_info = (ratings.shape[1], ratings.shape[0])
tags_info = (tags.shape[1], tags.shape[0])
links_info = (links.shape[1], links.shape[0])

print(f"Movies: {movies_info[1]} records, {movies_info[0]} attributes")
print(f"Ratings: {ratings_info[1]} records, {ratings_info[0]} attributes")
print(f"Tags: {tags_info[1]} records, {tags_info[0]} attributes")
print(f"Links: {links_info[1]} records, {links_info[0]} attributes")

In [ ]:
ratings_df = reviews.groupby(by=['rating']).agg({'userId': 'count'}).rename(columns={'userId': 'count'}).reset_index()
plt.bar(ratings_df['rating'], ratings_df['count'], align='center', width=0.3)
plt.xticks(ratings_df['rating'])
plt.title("Rating vs Count")
plt.xlabel("Rating")
plt.ylabel("Count")

In [ ]:
display(movies)

In [ ]:
vectorizer = TfidfVectorizer()
tag_matrix = vectorizer.fit_transform(reviews['tag'].fillna(''))

X = tag_matrix.toarray()
y = reviews['rating']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
models = {
    'DecisionTree': DecisionTreeRegressor(),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingRegressor(),
    'LinearRegression': LinearRegression()
}

In [ ]:
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = {'MSE': mse, 'R²': r2}
    print(f'{name} - MSE: {mse}, R²: {r2}')

In [ ]:
model = models['RandomForest']

In [ ]:
def prepare_features(new_movie_tags, tfidf):
    tag_features = tfidf.transform([new_movie_tags]).toarray()
    return tag_features

In [ ]:
new_movie_tags = ["action packed, thrilling", "romantic, comedy", "suspenseful, mysterious"]
new_movie_features = [prepare_features(tags, vectorizer) for tags in new_movie_tags]

In [ ]:
for tags, features in zip(new_movie_tags, new_movie_features):
    predicted_rating = model.predict(features)
    print(f'Predicted Rating for "{tags}": {predicted_rating[0]}')

In [ ]:
explainer = shap.Explainer(model)
shap_values = explainer(X_test, check_additivity=False)
for i in tqdm(range(len(X_test)), desc="Calculating SHAP values"):
    shap_values.append(explainer(X_test[i], check_additivity=False))
shap_values = np.array(shap_values)

In [ ]:
shap_values_array = np.vstack(shap_values)

print(shap_values_array.shape)

In [ ]:
shap.summary_plot(shap_values, X_test, feature_names=vectorizer.get_feature_names_out())

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0], X_test[0])